In [7]:
search_for = 120
start_from = 133819999
threads = 5
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004547834396362305
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 875
all cases: 1116151311
type: [1, 1, 1, 1, 120] 120
cases of this type: 207360000
50000 0.21488404560316401
100000 0.21165032711615167
150000 0.19948119708980322
200000 0.20153129583512677
250000 0.19966286115530332
300000 0.19762757930191754
350000 0.19480178541471826
400000 0.19502508111106556
450000 0.2026831656115214
500000 0.19582035518467958
550000 0.19489247018815672
600000 0.19892054093157452
650000 0.20166722726434996
700000 0.19571433247847822
750000 0.19527859036663772
800000 0.1976099368077795
850000 0.19518426486634544
900000 0.1969156603153984
950000 0.20496757019143635
1000000 0.19536628100703293
1050000 0.1944092842725436
1100000 0.1939825935337954
1150000 0.19781967083864083
1200000 0.19941313468658925
1250000 0.1957182491515398
1300000 0.19072865139122935
1350000 0.1887222474896749
1400000 0.18889687421434723
1450000 0.192

14300000 0.18516684006560374
14350000 0.2197678322556006
14400000 0.2194368082732081
14450000 0.19399639723958176
14500000 0.1962712246691876
14550000 0.16184159465920528
14600000 0.1959995326208578
14650000 0.19667052589719694
14700000 0.18277701158616544
14750000 0.1271759924443894
14800000 0.1898486891916739
14850000 0.18982840048520166
14900000 0.19131782578198112
14950000 0.18956028564621477
15000000 0.174999950857989
15050000 0.17881608981088265
15100000 0.19882006529438234
15150000 0.19427125936728717
15200000 0.19380600072876877
15250000 0.19687982586486602
15300000 0.19646844022027649
15350000 0.19375189226152234
15400000 0.19787505689592755
15450000 0.22231849327760136
15500000 0.19021495157178905
15550000 0.1959660739799155
15600000 0.11578624789108038
15650000 0.2081926666960266
15700000 0.15731587554826737
15750000 0.18786447018704414
15800000 0.18712720012918047
15850000 0.1840494094437308
15900000 0.18852963858025076
15950000 0.1859446192860312
16000000 0.145781487166963

28850000 0.20513670197621717
28900000 0.1869674036297639
28950000 0.18654263102429308
29000000 0.1921672683348404
29050000 0.19059467981989517
29100000 0.18761550045797823
29150000 0.21779340414517193
29200000 0.200241851990917
29250000 0.20730565399602652
29300000 0.18347096315094766
29350000 0.18487901048268343
29400000 0.18844754779454867
29450000 0.20114435262557268
29500000 0.18422594938204687
29550000 0.19800841618040801
29600000 0.19497556873034905
29650000 0.20930025930748516
29700000 0.20668449606150388
29750000 0.20441001339602605
29800000 0.2063388364775684
29850000 0.2199107189566652
29900000 0.2017818939154466
29950000 0.20193662061868511
30000000 0.2021819677503705
30050000 0.20189788923128973
30100000 0.20660552374489177
30150000 0.20257263795917035
30200000 0.21304632885913585
30250000 0.20859927764603697
30300000 0.21068090543377796
30350000 0.2065996725397269
30400000 0.20937437728493477
30450000 0.21905871895064113
30500000 0.2080954081713253
30550000 0.2179615321707

43300000 0.19933534304196837
43350000 0.18006045410987537
43400000 0.18670760560750962
43450000 0.18333434470620683
43500000 0.1828654672519207
43550000 0.1822261473259714
43600000 0.18210542175575758
43650000 0.20267402071472007
43700000 0.18658578680775034
43750000 0.1842148849976963
43800000 0.1848795564206203
43850000 0.18838905439076026
43900000 0.1874216026793639
43950000 0.1845540372077107
44000000 0.19700694451038836
44050000 0.19870039614263507
44100000 0.2058805034945647
44150000 0.20956836474354001
44200000 0.20775395999594662
44250000 0.20523267097959522
44300000 0.21497940078955757
44350000 0.2044625285124156
44400000 0.1834033735919595
44450000 0.18087658981340593
44500000 0.18341748906968303
44550000 0.20046002752586206
44600000 0.1812728118079212
44650000 0.1953873554654373
44700000 0.19410380688876708
44750000 0.19146937301613356
44800000 0.18847474399286773
44850000 0.2073782418782115
44900000 0.1853894393161999
44950000 0.19916072742294869
45000000 0.2039370132064024

57700000 0.19840668245516882
57750000 0.17979125358413062
57800000 0.17953323311245445
57850000 0.18670181646888784
57900000 0.18455679829137323
57950000 0.18074122939301862
58000000 0.18064298877391022
58050000 0.21927676587162812
58100000 0.20263271994879378
58150000 0.20153007313092416
58200000 0.18599687080958288
58250000 0.18738791984985936
58300000 0.18592464415127968
58350000 0.1923133165905714
58400000 0.21198746577034658
58450000 0.20191003301757046
58500000 0.18491025698558888
58550000 0.1900706273720172
58600000 0.20197244926008384
58650000 0.18305190249023837
58700000 0.1836637331496451
58750000 0.20824780560686457
58800000 0.19216427986133097
58850000 0.18953775926702685
58900000 0.18652218065968354
58950000 0.19285537952165202
59000000 0.20700695940255587
59050000 0.18310167847669126
59100000 0.1831028187257012
59150000 0.18146310107089678
59200000 0.1856642642465022
59250000 0.182124892681396
59300000 0.18127496688797212
59350000 0.2232291066254695
59400000 0.18638357992

...

best so far: 0
type: [2, 3, 10, 2, 1] 120
cases of this type: 2400
best so far: 0
type: [2, 3, 20, 1, 1] 120
cases of this type: 1200
best so far: 0
type: [2, 4, 1, 1, 15] 120
cases of this type: 202500
1101750000 0.34146075656762126
1101800000 0.5270932107777331
1101850000 0.2172116823835823
1101900000 0.3329290775486589
best so far: 0
type: [2, 4, 1, 3, 5] 120
cases of this type: 67500
1101950000 0.46687987588593693
1102000000 0.2523036060521576
best so far: 0
type: [2, 4, 1, 5, 3] 120
cases of this type: 40500
best so far: 0
type: [2, 4, 1, 15, 1] 120
cases of this type: 13500
1102050000 0.2409942073262374
best so far: 0
type: [2, 4, 3, 1, 5] 120
cases of this type: 22500
best so far: 0
type: [2, 4, 3, 5, 1] 120
cases of this type: 4500
best so far: 0
type: [2, 4, 5, 1, 3] 120
cases of this type: 8100
best so far: 0
type: [2, 4, 5, 3, 1] 120
cases of this type: 2700
best so far: 0
type: [2, 4, 15, 1, 1] 120
cases of this type: 900
best so far: 0
type: [2, 5, 1, 1, 12] 120
cases of t

1107800000 0.22457670588608583
1107850000 0.22084787496767974
1107900000 0.22478911802912158
best so far: 0
type: [3, 1, 1, 10, 4] 120
cases of this type: 256000
1107950000 0.24674282399365902
1108000000 0.16648232532115645
1108050000 0.24245082677175997
1108100000 0.22096864427396984
1108150000 0.13405264546413156
best so far: 0
type: [3, 1, 1, 20, 2] 120
cases of this type: 128000
1108200000 0.0920028767098546
1108250000 0.06056144282219145
1108300000 0.06034197646128866
best so far: 0
type: [3, 1, 1, 40, 1] 120
cases of this type: 64000
1108350000 0.06302310506386757
best so far: 0
type: [3, 1, 2, 1, 20] 120
cases of this type: 640000
1108400000 0.16644585016036298
1108450000 0.17703884034320647
1108500000 0.19505460238348646
1108550000 0.2863027764407701
1108600000 0.3493805031623642
1108650000 0.17069208585709333
1108700000 0.06661762203885184
1108750000 0.0652666712652074
1108800000 0.11022176984856129
1108850000 0.19860228007643352
1108900000 0.15136086621328196
1108950000 0.195

1112650000 0.05873708944475783
1112700000 0.08261212113318445
1112750000 0.052161026412651273
best so far: 0
type: [4, 1, 1, 5, 6] 120
cases of this type: 162000
1112800000 0.07259238619471389
1112850000 0.05389352468972206
1112900000 0.06442757804810471
1112950000 0.06746694992893536
best so far: 0
type: [4, 1, 1, 6, 5] 120
cases of this type: 135000
1113000000 0.039849987475717066
1113050000 0.0496606925131268
best so far: 0
type: [4, 1, 1, 10, 3] 120
cases of this type: 81000
1113100000 0.03464361118444072
1113150000 0.0670949837538004
best so far: 0
type: [4, 1, 1, 15, 2] 120
cases of this type: 54000
1113200000 0.05498517937940492
best so far: 0
type: [4, 1, 1, 30, 1] 120
cases of this type: 27000
1113250000 0.03413809125632975
best so far: 0
type: [4, 1, 2, 1, 15] 120
cases of this type: 202500
1113300000 0.04461002014470498
1113350000 0.03547472700915071
1113400000 0.028120549840863547
1113450000 0.045137237411169207
best so far: 0
type: [4, 1, 2, 3, 5] 120
cases of this type: 6

best so far: 0
type: [5, 2, 2, 2, 3] 120
cases of this type: 5184
best so far: 0
type: [5, 2, 2, 3, 2] 120
cases of this type: 3456
best so far: 0
type: [5, 2, 2, 6, 1] 120
cases of this type: 1728
best so far: 0
type: [5, 2, 3, 1, 4] 120
cases of this type: 4608
1115400000 0.010285541216635705
best so far: 0
type: [5, 2, 3, 2, 2] 120
cases of this type: 2304
best so far: 0
type: [5, 2, 3, 4, 1] 120
cases of this type: 1152
best so far: 0
type: [5, 2, 4, 1, 3] 120
cases of this type: 2592
best so far: 0
type: [5, 2, 4, 3, 1] 120
cases of this type: 864
best so far: 0
type: [5, 2, 6, 1, 2] 120
cases of this type: 1152
best so far: 0
type: [5, 2, 6, 2, 1] 120
cases of this type: 576
best so far: 0
type: [5, 2, 12, 1, 1] 120
cases of this type: 288
best so far: 0
type: [5, 3, 1, 1, 8] 120
cases of this type: 12288
best so far: 0
type: [5, 3, 1, 2, 4] 120
cases of this type: 6144
best so far: 0
type: [5, 3, 1, 4, 2] 120
cases of this type: 3072
best so far: 0
type: [5, 3, 1, 8, 1] 120
case

best so far: 0
type: [10, 2, 1, 2, 3] 120
cases of this type: 1296
best so far: 0
type: [10, 2, 1, 3, 2] 120
cases of this type: 864
best so far: 0
type: [10, 2, 1, 6, 1] 120
cases of this type: 432
best so far: 0
type: [10, 2, 2, 1, 3] 120
cases of this type: 648
best so far: 0
type: [10, 2, 2, 3, 1] 120
cases of this type: 216
best so far: 0
type: [10, 2, 3, 1, 2] 120
cases of this type: 288
best so far: 0
type: [10, 2, 3, 2, 1] 120
cases of this type: 144
best so far: 0
type: [10, 2, 6, 1, 1] 120
cases of this type: 72
best so far: 0
type: [10, 3, 1, 1, 4] 120
cases of this type: 768
best so far: 0
type: [10, 3, 1, 2, 2] 120
cases of this type: 384
best so far: 0
type: [10, 3, 1, 4, 1] 120
cases of this type: 192
best so far: 0
type: [10, 3, 2, 1, 2] 120
cases of this type: 192
best so far: 0
type: [10, 3, 2, 2, 1] 120
cases of this type: 96
best so far: 0
type: [10, 3, 4, 1, 1] 120
cases of this type: 48
best so far: 0
type: [10, 4, 1, 1, 3] 120
cases of this type: 324
best so far: